<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Demo map data preparation

_by Joe Hamman & Jeremy Freeman (CarbonPlan), September 27, 2021_

This notebook demonstrates the production of Zarr data pyramids for use with
[`@carbonplan/maps`](https://github.com/carbonplan/maps), an api for interactive
multi-dimensional data-driven web maps.

Some of the libraries used here are in pre-release condition. Specifically
`ndpyramid` and `datatree` are currently udergoing rapid development. Use the
pattern below but expect changes to the specific apis.


In [ ]:
import xarray as xr
import pandas as pd
import rioxarray
from ndpyramid import pyramid_reproject
from carbonplan_data.utils import set_zarr_encoding
from carbonplan_data.metadata import get_cf_global_attrs

In [ ]:
VERSION = 2
LEVELS = 6
PIXELS_PER_TILE = 128

input_path = f"gs://carbonplan-maps/v{VERSION}/demo/raw"
save_path = f"gs://carbonplan-maps/v{VERSION}/demo/"

## 2d (tavg)

In this example we open a single 2d image (GeoTIFF) and create a Zarr pyramid.


In [ ]:
%%time
# input dataset
path = f"{input_path}/wc2.1_2.5m_tavg_10.tif"

# open and extract the input dataset
ds = (
    xr.open_dataarray(path, engine="rasterio")
    .to_dataset(name="tavg")
    .squeeze()
    .reset_coords(["band"], drop=True)
)

# create the pyramid
dt = pyramid_reproject(ds, levels=LEVELS)

# modify the data in the pyramid
for child in dt.children.values():
    child.ds = set_zarr_encoding(
        child.ds, codec_config={"id": "zlib", "level": 1}, float_dtype="float32"
    )
    child.ds = child.ds.chunk({"x": PIXELS_PER_TILE, "y": PIXELS_PER_TILE})
    child.ds["tavg"].attrs.clear()
dt.attrs = get_cf_global_attrs(version=VERSION)

for level in range(LEVELS):
    slevel = str(level)
    dt.ds.attrs['multiscales'][0]['datasets'][level]['pixels_per_tile'] = PIXELS_PER_TILE
dt.ds.attrs['multiscales'][0]['metadata']['version'] = VERSION

# write the pyramid to zarr
dt.to_zarr(save_path + "2d/tavg", consolidated=True)

## 3d, two variables (tavg and prec)

In this example, we open two 2d images (temperature and precipitation), combine
them into a single array (along the `band` dimension), and create a Zarr
pyramid.


In [ ]:
%%time
# input datasets
path1 = f"{input_path}/wc2.1_2.5m_tavg_10.tif"
path2 = f"{input_path}/wc2.1_2.5m_prec_10.tif"

# open and extract the input datasets
ds1 = (
    xr.open_dataarray(path1, engine="rasterio")
    .to_dataset(name="climate")
    .squeeze()
    .reset_coords(["band"], drop=True)
)
ds2 = (
    xr.open_dataarray(path2, engine="rasterio")
    .to_dataset(name="climate")
    .squeeze()
    .reset_coords(["band"], drop=True)
)
ds2["climate"] = ds2["climate"].astype("float32")
ds2["climate"].values[ds2["climate"].values == ds2["climate"].values[0, 0]] = ds1["climate"].values[
    0, 0
]
ds = xr.concat([ds1, ds2], pd.Index(["tavg", "prec"], name="band"))
ds["band"] = ds["band"].astype("str")

# create the pyramid
dt = pyramid_reproject(ds, levels=LEVELS)

# modify the data in the pyramid
for child in dt.children.values():
    child.ds = set_zarr_encoding(
        child.ds, codec_config={"id": "zlib", "level": 1}, float_dtype="float32"
    )
    child.ds = child.ds.chunk({"x": PIXELS_PER_TILE, "y": PIXELS_PER_TILE, "band": 2})
    child.ds["climate"].attrs.clear()
dt.attrs = get_cf_global_attrs(version=VERSION)

for level in range(LEVELS):
    slevel = str(level)
    dt.ds.attrs['multiscales'][0]['datasets'][level]['pixels_per_tile'] = PIXELS_PER_TILE
dt.ds.attrs['multiscales'][0]['metadata']['version'] = VERSION

# write the pyramid to zarr
dt.to_zarr(save_path + "3d/tavg-prec", consolidated=True)
dt.ds.attrs

## 3d, one variable, multiple time points

In this example, we open 12 2d images (one map of temperature for each month),
combine them into a single array (along the `month` dimension), and create a
Zarr pyramid.


In [ ]:
%%time
# open and extract the input datasets
ds_all = []
months = list(map(lambda d: d + 1, range(12)))
for i in months:
    path = f"{input_path}/wc2.1_2.5m_tavg_{i:02g}.tif"
    ds = (
        xr.open_dataarray(path, engine="rasterio")
        .to_dataset(name="tavg")
        .squeeze()
        .reset_coords(["band"], drop=True)
    )
    ds_all.append(ds)
ds = xr.concat(ds_all, pd.Index(months, name="month"))
ds["month"] = ds["month"].astype("int32")

# create the pyramid
dt = pyramid_reproject(ds, levels=LEVELS)

# modify the data in the pyramid
for child in dt.children.values():
    child.ds = set_zarr_encoding(
        child.ds, codec_config={"id": "zlib", "level": 1}, float_dtype="float32"
    )
    child.ds = child.ds.chunk({"x": PIXELS_PER_TILE, "y": PIXELS_PER_TILE, "month": 12})
    child.ds["tavg"].attrs.clear()
dt.attrs = get_cf_global_attrs(version=VERSION)

for level in range(LEVELS):
    slevel = str(level)
    dt.ds.attrs['multiscales'][0]['datasets'][level]['pixels_per_tile'] = PIXELS_PER_TILE
dt.ds.attrs['multiscales'][0]['metadata']['version'] = VERSION


# write the pyramid to zarr
dt.to_zarr(save_path + "3d/tavg-month", consolidated=True)
dt.ds.attrs

## 4d, multiple variables, multiple time points

In this example, we open 12 2d images for 2 variables (one map of temperature
and precipitation for each month), combine them into a single array (along the
`month` and `band` dimensions), and create a Zarr pyramid.


In [ ]:
%%time
# open and extract the input datasets
ds1_all = []
ds2_all = []
months = list(map(lambda d: d + 1, range(12)))
for i in months:
    path = f"{input_path}/wc2.1_2.5m_tavg_{i:02g}.tif"
    ds = (
        xr.open_dataarray(path, engine="rasterio")
        .to_dataset(name="climate")
        .squeeze()
        .reset_coords(["band"], drop=True)
    )
    ds1_all.append(ds)
ds1 = xr.concat(ds1_all, pd.Index(months, name="month"))
for i in months:
    path = f"{input_path}/wc2.1_2.5m_prec_{i:02g}.tif"
    ds = (
        xr.open_dataarray(path, engine="rasterio")
        .to_dataset(name="climate")
        .squeeze()
        .reset_coords(["band"], drop=True)
    )
    ds2_all.append(ds)
ds2 = xr.concat(ds2_all, pd.Index(months, name="month"))
ds1["month"] = ds1["month"].astype("int32")
ds2["month"] = ds2["month"].astype("int32")
ds2["climate"] = ds2["climate"].astype("float32")
ds2["climate"].values[ds2["climate"].values == ds2["climate"].values[0, 0, 0]] = ds1[
    "climate"
].values[0, 0, 0]
ds = xr.concat([ds1, ds2], pd.Index(["tavg", "prec"], name="band"))
ds["band"] = ds["band"].astype("str")

# create the pyramid
dt = pyramid_reproject(ds, levels=LEVELS, extra_dim="band")
for child in dt.children.values():
    child.ds = set_zarr_encoding(
        child.ds, codec_config={"id": "zlib", "level": 1}, float_dtype="float32"
    )
    child.ds = child.ds.chunk({"x": PIXELS_PER_TILE, "y": PIXELS_PER_TILE, "band": 2, "month": 12})
    child.ds["climate"].attrs.clear()
dt.attrs = get_cf_global_attrs(version=VERSION)

for level in range(LEVELS):
    slevel = str(level)
    dt.ds.attrs['multiscales'][0]['datasets'][level]['pixels_per_tile'] = PIXELS_PER_TILE
dt.ds.attrs['multiscales'][0]['metadata']['version'] = VERSION


# write the pyramid to zarr
dt.to_zarr(save_path + "4d/tavg-prec-month", consolidated=True)
dt.ds.attrs